## 构建Two-Head网络对OCSVM进行改进

## 数据预处理

In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from tqdm import tqdm
import visdom

from utils import *

# tensorflowGPU的设置

# tf.config.experimental.list_physical_devices('CPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' #屏蔽通知信息，输出warning, Error, fatal 

In [2]:
# 加载数据
def judge_1(time):
    time = time[:7].replace('-', '')
    if time <= '201803':
        return True
    else:
        return False

def judge_2(time):
    time = time[:7].replace('-', '')
    if time > '201803':
        return True
    else:
        return False

name2label = {'trojan':0, 'virus':1, 'worm':2, 'backdoor':3}
data_csv = pd.read_csv('res_handle.csv')
print(data_csv.describe())
data_2017 = data_csv[data_csv['first_seen'].apply(lambda x: x[:4]) == '2017']
data_2018 = data_csv[data_csv['first_seen'].apply(lambda x: x[:4]) == '2018']
data_2019 = data_csv[data_csv['first_seen'].apply(lambda x: x[:4]) == '2019']
# 将201803之前的数据划分到2017数据集下    
data_2018_1 = data_2018[data_2018['first_seen'].apply(judge_1)]
data_2018_2 = data_2018[data_2018['first_seen'].apply(judge_2)]

data_train = data_2017.append(data_2018_１)
data_test_1 = data_2018_２
data_test_2 = data_2019
print("未采样前数据集的规模：", data_train.shape, data_test_1.shape, data_test_2.shape)

# 对测试集进行下采样
data_train = data_train.sample(frac=1, random_state=1).reset_index(drop=True)
data_test_1 = data_test_1.sample(frac=0.7, random_state=1).reset_index(drop=True)
data_test_2 = data_test_2.sample(frac=0.5, random_state=1).reset_index(drop=True)
print("采样后数据集的规模：", data_train.shape, data_test_1.shape, data_test_2.shape)

#获取数据的路径及标签
data_train_names, labels_train = name_loader(data_train)
data_test_1_names, labels_test_1 = name_loader(data_test_1)
data_test_2_names, labels_test_2 = name_loader(data_test_2)

              length
count   12402.000000
mean     5652.612804
std     20251.140030
min         2.000000
25%       202.000000
50%      1105.500000
75%      2849.000000
max    780386.000000
未采样前数据集的规模： (6104, 5) (2642, 5) (3656, 5)
采样后数据集的规模： (6104, 5) (1849, 5) (1828, 5)


In [8]:
# 利用labelencoder构建opcode编码器
opcode_all = []
for idx in tqdm(range(data_csv.shape[0])):
    opcode_str = data_csv.at[idx, 'opcode'].split()
    opcode_all += opcode_str
    opcode_all = list(set(opcode_all))

le = preprocessing.LabelEncoder()
le.fit(opcode_all)
len(list(le.classes_))

100%|██████████| 12402/12402 [00:07<00:00, 1663.09it/s]


641

## 训练two-head模型

In [4]:
# 创建TextCNN类
from tensorflow import keras
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout

class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=4,
                 last_activation=None):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        # Embedding part can try multichannel as same as origin paper
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        convs = []
        for kernel_size in [2, 3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)
        d_1 = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
        d_2 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(d_1)

        output_1 = Dense(self.class_num, activation=self.last_activation)(d_2)
        output_2 = Dense(self.class_num, activation=self.last_activation)(d_2)
        model = Model(inputs=input, outputs=[output_1, output_2])
        return model

In [5]:
def pad_data(res, max_len=2850):
    length = len(res)
    if length > max_len:
        return res[ :max_len]
    elif length < max_len:
        return res + [0]*(max_len-length)
    return res


def train_data_generator(codes, labels, mode):
    
    if mode == 'train':    
        codes = codes[: 5100]
        labels = labels[: 5100]
    elif mode == 'val':    
        codes = codes[5100: ]
        labels = labels[5100: ]

    labels_res = np.eye(4)[labels]
    for idx in range(len(codes)):
        fn = codes[idx]
        fn = bytes.decode(fn)
        op_string = data_csv[data_csv.name == fn]
        if len(op_string) == 0:
            print('null')
            continue
        op_string = op_string.iloc[0, 1].split()
        res = list(le.transform(op_string))
        res = pad_data(res)
        yield res, labels_res[idx]
        
        
def train_data_loader(codes, labels, mode):
    
    if mode == 'train':    
        codes = codes[: 5100]
        labels = labels[: 5100]
    elif mode == 'val':    
        codes = codes[5100: ]
        labels = labels[5100: ]
    
    labels_res = np.eye(4)[labels]
    res_all = []
    for idx in tqdm(range(len(codes))):
        fn = codes[idx]
        op_string = data_csv[data_csv.name == fn]
        op_string = op_string.iloc[0, 1].split()
        res = list(le.transform(op_string))
        res = pad_data(res)
        res_all.append(res)
    return np.array(res_all), labels_res


def codes_loader(codes):
    
    res_all = []
    for idx in tqdm(range(len(codes))):
        fn = codes[idx]
        op_string = data_csv[data_csv.name == fn]
        op_string = op_string.iloc[0, 1].split()
        res = list(le.transform(op_string))
        res = pad_data(res)
        res_all.append(res)
    return np.array(res_all)

In [7]:
from tensorflow.keras import Input, Model, regularizers
from tensorflow import keras

# 实例化TextCNN模型
max_len = 2850
max_features = 641
embedding_dims = 16
model = TextCNN(max_len, max_features, embedding_dims).get_model()

# 创建dataset对象
batch_size = 128
max_len = 2850
db_train = tf.data.Dataset.from_generator(train_data_generator, args=[data_train_names, labels_train, 'train'], 
                                          output_types=(tf.float32, tf.int32), output_shapes=((None,), (4, )))
db_val = tf.data.Dataset.from_generator(train_data_generator, args=[data_train_names, labels_train, 'val'], 
                                          output_types=(tf.float32, tf.int32), output_shapes=((None,), (4, )))

# 创建训练过程
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_sup = keras.losses.CategoricalCrossentropy(from_logits=True)
metrics_train_1 = keras.metrics.CategoricalAccuracy()
metrics_train_2 = keras.metrics.CategoricalAccuracy()
metrics_val_1 = keras.metrics.CategoricalAccuracy()
metrics_val_2 = keras.metrics.CategoricalAccuracy()

epochs = 40
db_train = db_train.batch(batch_size)
db_val = db_val.batch(batch_size)

loss_all = []
acc_all = []

for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))
    
    for step, (x_batch_train, y_batch_train) in enumerate(db_train):
        with tf.GradientTape() as tape:
            out1, out2 = model(x_batch_train)
            loss_value = loss_sup(y_batch_train, out1) + loss_sup(y_batch_train, out2)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        metrics_train_1(y_batch_train, out1)
        metrics_train_2(y_batch_train, out2)
        
        if step % 10 == 0:
            loss_all.append(loss_value)
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
    train_acc_1 = metrics_train_1.result()
    train_acc_2 = metrics_train_2.result()
    print('Training acc over epoch: %s %s' % (float(train_acc_1), float(train_acc_2)))
    metrics_train_1.reset_states()
    metrics_train_2.reset_states()
    
    for x_batch_val, y_batch_val in db_val:
        val_out_1, val_out_2 = model(x_batch_val)
        metrics_val_1(y_batch_val, val_out_1)
        metrics_val_2(y_batch_val, val_out_2)
    
    val_acc_1 = metrics_val_1.result()
    val_acc_2 = metrics_val_2.result()
    acc_all.append(val_acc_1)
    print('Validation acc: %s %s' % (float(val_acc_1), float(val_acc_2))) 
    metrics_val_1.reset_states()
    metrics_val_2.reset_states()

Start of epoch 0
Training loss (for one batch) at step 0: 2.7653419971466064
Training loss (for one batch) at step 10: 2.726194381713867
Training loss (for one batch) at step 20: 2.5980563163757324
Training loss (for one batch) at step 30: 2.2676656246185303
Training loss (for one batch) at step 40: 2.0087127685546875
Training acc over epoch: 0.4954128563404083 0.4906618595123291
Validation acc: 0.7187090516090393 0.6587483882904053
Start of epoch 1
Training loss (for one batch) at step 0: 1.6196199655532837
Training loss (for one batch) at step 10: 1.7780039310455322
Training loss (for one batch) at step 20: 1.352810263633728
Training loss (for one batch) at step 30: 0.895261824131012
Training loss (for one batch) at step 40: 1.026787281036377
Training acc over epoch: 0.7961992025375366 0.8006225228309631
Validation acc: 0.8648427128791809 0.8682830929756165
Start of epoch 2
Training loss (for one batch) at step 0: 0.7006429433822632
Training loss (for one batch) at step 10: 1.0168956

In [ ]:
# 保存模型
model.save('./model/textcnn_twohead.h5')

In [ ]:
# 绘制损失函数曲线
p1 = plt.figure(figsize=(5, 3.5),dpi=300)
plt.plot(loss_all)
plt.title("textcnn loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train"],loc="upper right")
plt.savefig("./figures/" + "twohead_loss.png")

In [17]:
from tensorflow.keras import Input, Model, regularizers
from tensorflow import keras
from utils import *

max_len = 2850
max_features = 641
embedding_dims = 16
model = TextCNN(max_len, max_features, embedding_dims).get_model()
model.load_weights('./model/textcnn_twohead.h5')

data_test_1 = pd.read_csv('data_test_1.csv')
ood2label = {True: 0, False: 1}
labels_test_1 = data_test_1['id'].map(lambda x: ood2label[x])
labels_test_1 = labels_test_1.to_list()
names_test_1 = data_test_1['name'].to_list()

codes_test_1 = codes_loader(data_test_1_names)
labels_test_1 = np.array(labels_test_1)
test_1_auc, discs_1 = get_auc(model, codes_test_1, labels_test_1)
print('Test AUROC: {:.3f}'.format(test_1_auc))
plot_entropy(discs_1, labels_test_1, 'entropy_distribution_test1.png')

100%|██████████| 1849/1849 [00:13<00:00, 137.63it/s]
Test AUROC: 0.923


In [ ]:
data_test_2 = pd.read_csv('data_test_2.csv')
ood2label = {True: 0, False: 1}
labels_test_2 = data_test_2['id'].map(lambda x: ood2label[x])
labels_test_2 = labels_test_2.to_list()
paths_test_2 = data_test_2['name'].to_list()
codes_test_2 = codes_loader(data_test_2_names)
labels_test_2 = np.array(labels_test_2)
test_1_auc, discs_2 = get_auc(model, codes_test_2, labels_test_2)
print('Test AUROC: {:.3f}'.format(test_1_auc))
plot_entropy(discs_2, labels_test_２, 'entropy_distribution_test2.png')

## 保存利用two-head异常检测模型得到的数据方便进行重训练

In [54]:
from sklearn.metrics import roc_curve, auc

# 确定异常检测阈值
fpr, tpr, thresholds = roc_curve(labels_test_1, discs_1)
roc_auc = auc(fpr, tpr)
for i in range(len(fpr)):
    if fpr[i] + tpr[i] >= 1:
        i = i - 1
        break
print(fpr[i])
score = thresholds[i]
print("score: ", score)

bool_ = discs_1 >= score
# print(type(bool_))
# for i in bool_.tolist():
#     if i == True:
#         print(i)
print(len(data_test_1_names), len(bool_.tolist()))
data_ood_names = [item for item, value in zip(data_test_1_names, bool_) if value == True] 

data_ood = data_test_1[data_test_1.name.isin(data_ood_names)]

data_ood = data_ood.reset_index(drop=True)
print(data_ood.head(5))
print(data_test_1.head(5))


data_ood.to_csv("oodFromTwoHead.csv", index=None)

0.16262135922330098
score:  0.0014142394
1849 1849
                                                name  \
0  3fd80e5c6326b1debfb100402521cf1b7e6dd7cb5a8bcf...   
1  51f19f2d2720a6533c0ce9ee72d0235f28e87531937f15...   
2  4bcb1ba52a53b54dab1eef63abb56c19ce989e782d5060...   
3  d7af87c0c1535f87196af6a00aa620073e36f25201f4a1...   
4  b0ecefe0e9c334635c6f94b9d26963f41a668c8fcd748b...   

                                              opcode  length     label  \
0  push mov add push mov retn mov push mov cmp jz...     203    trojan   
1  call ldc.i4 nop ldc.i4 stloc.1 nop stloc.2 nop...   68302  backdoor   
2  push add mov push call test jz movzx mov add p...    6740      worm   
3                              mov cmp mov push retn       5      worm   
4  nop call nop ret ldloca.s call stfld ldloca.s ...     199    trojan   

   first_seen  novelty     id  
0  2018-06-19        1   True  
1  2018-06-18        1  False  
2  2018-06-19        0  False  
3  2018-06-19        1   True  
4  2018

In [ ]:
from sklearn.metrics import classification_report

labels_pre_id = data_test_1['label'].map(lambda x: name2label[x])
labels_pre_id = np.array(labels_pre_id).reshape(-1, 1)
labels_pre_id = labels_pre_id[discs<thresholds[i]]

model_pre = keras.models.load_model('./model/textcnn_split.h5')
bool_code = discs<thresholds[i]
codes_pre_id = codes_test_1[bool_code.reshape(-1)]
y_pred = model_pre.predict(codes_pre_id)
y_pred = np.argmax(y_pred, 1)
target_names = {'trojan', 'virus', 'worm', 'backdoor'}
print(classification_report(labels_pre_id, y_pred, target_names=target_names, digits=4))

## 进行参数微调并改进模型

In [ ]:
# 自定义损失函数
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model, regularizers
from tensorflow import keras

model = TextCNN(max_len, max_features, embedding_dims).get_model()
class DiscrepancyLoss(keras.losses.Loss):
    def __init__(self, reduction=keras.losses.Reduction.NONE, 
                 from_logits=True, name='membership loss'):
        super().__init__(reduction=reduction, name=name)
    
    def call(self, out1, out2, m=0.3):
        out_1 = K.softmax(out1)
        out_2 = K.softmax(out2)     
        entropy_1 = -out_1 * K.log(out_1)
        entropy_2 = -out_2 * K.log(out_2)
        entropy_1 = K.mean(entropy_1, axis=1)
        entropy_2 = K.mean(entropy_2, axis=1) 
        
        return K.relu(m - K.mean(entropy_1 - entropy_2))

In [ ]:
# 加载SVM预测数据
data_test_1 = pd.read_csv('data_test_1.csv')
data_test_unsup = data_test_1[data_test_1['novelty']!=0]
codes_unsup = data_test_unsup['name'].to_list()

In [ ]:
data_test_unsup.shape

In [ ]:
def codes_generator(codes):
    
    for idx in range(len(codes)):
        fn = codes[idx]
        fn = bytes.decode(fn)
        op_string = data_csv[data_csv.name == fn]
        if len(op_string) == 0:
            print('null')
            continue
        op_string = op_string.iloc[0, 1].split()
        res = list(le.transform(op_string))
        res = pad_data(res)
        yield res

def svmdata_generator(codes, labels, mode):
    
    if mode == 'train':    
        codes = codes[: 522]
        labels = labels[: 522]
    elif mode == 'val':    
        codes = codes[522: 1000]
        labels = labels[522: 1000]
    
    labels_res = np.eye(4)[labels]
    for idx in range(len(codes)):
        fn = codes[idx]
        fn = bytes.decode(fn)
        op_string = data_csv[data_csv.name == fn]
        if len(op_string) == 0:
            print('null')
            continue
        op_string = op_string.iloc[0, 1].split()
        res = list(le.transform(op_string))
        res = pad_data(res)
        yield res, labels_res[idx]

In [ ]:
# 准备训练数据
batch_size = 64
sup_train = tf.data.Dataset.from_generator(svmdata_generator, args=[data_path_1, labels_1, 'train'], 
                                          output_types=(tf.float32, tf.int32), output_shapes=((None,), (4, )))
sup_train = sup_train.batch(batch_size)

sup_val = tf.data.Dataset.from_generator(svmdata_generator, args=[data_path_1, labels_1, 'val'], 
                                          output_types=(tf.float32, tf.int32), output_shapes=((None,), (4, )))
sup_val = sup_val.batch(batch_size)

unsup_train = tf.data.Dataset.from_generator(codes_generator, args=[codes_unsup], output_types=(tf.float32), output_shapes=((None,)))
unsup_train = unsup_train.batch(batch_size)

In [ ]:
# 准备测试数据
data_test_1 = pd.read_csv('data_test_1.csv')
ood2label = {True: 0, False: 1}
labels_test_1 = data_test_1['id'].map(lambda x: ood2label[x])
labels_test_1 = labels_test_1.to_list()
paths_test_1 = data_test_1['name'].to_list()
codes_test_1 = codes_loader(paths_test_1)
labels_test_1 = np.array(labels_test_1)

In [ ]:
# 开始训练
import visdom

model = TextCNN(max_len, max_features, embedding_dims).get_model()
model.load_weights('./model/textcnn_twohead.h5')
optimizer = keras.optimizers.Adam(learning_rate=0.00005)
loss_sup = keras.losses.CategoricalCrossentropy(from_logits=True)
loss_unsup = DiscrepancyLoss()
train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()
vis = visdom.Visdom(server='http://localhost')

print('>> Fine-tune a Model.')
best_auc = 0.
num_epochs = 10
iters = 0
plot_data = {'X': [], 'Y': [], 'legend': ['Sup. Loss', 'Unsup. Loss', 'Tot. Loss']}
for epoch in range(num_epochs):
    for step, batch in enumerate(zip(sup_train, unsup_train)):
        
        (x_batch_train, y_batch_train), (x_batch_train_unsup) = batch
#         print('Start of epoch %d step %d' % (epoch,step))
        with tf.GradientTape() as tape:
            out_1, out_2 = model(x_batch_train)
            loss_sup_value = (loss_sup(y_batch_train, out_1) + loss_sup(y_batch_train, out_2)) / 2
            out_1, out_2 = model(x_batch_train_unsup)
            loss_unsup_value = loss_unsup(out_1, out_2)
            loss_value = loss_sup_value + loss_unsup_value
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # 可视化
        if step % 2 == 0:
            
            loss_sup_value = loss_sup_value.numpy()
            loss_unsup_value = loss_unsup_value.numpy()
            loss_value = loss_value.numpy()
            
            plot_data['X'].append(iters)
            plot_data['Y'].append([
                loss_sup_value, loss_unsup_value, loss_value])
            vis.line(
                X=np.stack([np.array(plot_data['X'])] * len(plot_data['legend']), 1),
                Y=np.array(plot_data['Y']),
                opts={
                    'title': 'Loss over Time',
                    'legend': plot_data['legend'],
                    'xlabel': 'Iterations',
                    'ylabel': 'Loss',
                    'width': 900,
                    'height': 300,
                },
                win=2
            )
        iters += 1 
        
        # 进行验证
        if step % 2 == 0:
            out_1, out_2 = model.predict(codes_test_1)
            out_1 = keras.backend.softmax(out_1).numpy()
            out_2 = keras.backend.softmax(out_2).numpy()
            
            entropy_1 = np.max(out_1, axis=1)
            entropy_2 = np.max(out_2, axis=1)
            
#             entropy_1 = keras.backend.mean(entropy_1, axis=1)
#             entropy_2 = keras.backend.mean(entropy_2, axis=1)
        
            labels = np.array(labels_test_1)
            discs = np.abs(entropy_1 - entropy_2).reshape((-1, 1))
            test_auc = metrics.roc_auc_score(labels, discs)
            print('Test AUROC: {:.3f}'.format(test_auc))
            if test_auc > best_auc:
                best_auc = test_auc
                model.save('./model/twohead_best.h5')

In [ ]:
model.load_weights('./model/twohead_best.h5')

from sklearn import metrics
auc_res, discs = get_auc(model, codes_test_1, labels_test_1)
print('Final AUROC: {:.3f}'.format(auc_res))
plot_entropy(discs, labels_test_1, 'entropy_distribution_1.png')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(labels_test_1, discs)
roc_auc = auc(fpr, tpr)
for i in range(len(fpr)):
    if fpr[i] + tpr[i] >= 1:
        i = i - 1
        break
print(fpr[i])

# 画图，只需要plt.plot(fpr,tpr),变量roc_auc只是记录auc的值，通过auc()函数能计算出来
plt.plot(fpr, tpr, lw=1, label='ROC (area = %0.2f) threshold = %f' % ( roc_auc, thresholds[i]))
# 画对角线
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('twohead_roc')

pred = discs>thresholds[i]
pred = pred.astype(int)
from sklearn.metrics import accuracy_score
accuracy_score(labels_test_1, pred)

In [ ]:
from sklearn.metrics import classification_report

labels_pre_id = data_test_1['label'].map(lambda x: name2label[x])
labels_pre_id = np.array(labels_pre_id).reshape(-1, 1)
labels_pre_id = labels_pre_id[discs<thresholds[i]]

model_pre = keras.models.load_model('./model/textcnn_split.h5')
bool_code = discs<thresholds[i]
codes_pre_id = codes_test_1[bool_code.reshape(-1)]
y_pred = model_pre.predict(codes_pre_id)
y_pred = np.argmax(y_pred, 1)
target_names = {'trojan', 'virus', 'worm', 'backdoor'}
print(classification_report(labels_pre_id, y_pred, target_names=target_names, digits=4))

In [ ]:
print(classification_report(labels_test_1, pred, digits=4))

In [ ]:
# 对第二个子集进行测试

model.load_weights('./model/twohead_best.h5')
from sklearn import metrics

data_test_2 = pd.read_csv('data_test_2.csv')
ood2label = {True: 0, False: 1}
labels_test_2 = data_test_2['id'].map(lambda x: ood2label[x])
labels_test_2 = labels_test_2.to_list()
paths_test_2 = data_test_2['name'].to_list()
codes_test_2 = codes_loader(paths_test_2)
labels_test_2 = np.array(labels_test_2)
auc_res, discs = get_auc(model, codes_test_2, labels_test_2)
print('Final AUROC: {:.3f}'.format(auc_res))
plot_entropy(discs, labels_test_2, 'entropy_distribution_2.png')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(labels_test_2, discs)
roc_auc = auc(fpr, tpr)
for i in range(len(fpr)):
    if fpr[i] + tpr[i] >= 1:
        i = i - 1
        break
print(fpr[i])

# 画图，只需要plt.plot(fpr,tpr),变量roc_auc只是记录auc的值，通过auc()函数能计算出来
plt.plot(fpr, tpr, lw=1, label='ROC (area = %0.2f) threshold = %f' % ( roc_auc, thresholds[i]))
# 画对角线
plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('twohead_roc_2')

pred = discs>thresholds[i]
pred = pred.astype(int)
from sklearn.metrics import accuracy_score
accuracy_score(labels_test_2, pred)

In [ ]:
from sklearn.metrics import classification_report

labels_pre_id = data_test_2['label'].map(lambda x: name2label[x])
labels_pre_id = np.array(labels_pre_id).reshape(-1, 1)
labels_pre_id = labels_pre_id[discs<thresholds[i]]

model_pre = keras.models.load_model('./model/textcnn_split.h5')
bool_code = discs<thresholds[i]
codes_pre_id = codes_test_2[bool_code.reshape(-1)]
y_pred = model_pre.predict(codes_pre_id)
y_pred = np.argmax(y_pred, 1)
target_names = {'trojan', 'virus', 'worm', 'backdoor'}
print(classification_report(labels_pre_id, y_pred, target_names=target_names, digits=4))

In [ ]:
print(classification_report(labels_test_2, pred, digits=4))